In [13]:
import numpy as np
import pandas as pd
from os import getcwd
from sklearn import preprocessing
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
import os
import pickle
from collections import Counter

In [2]:
def set_path(): 
   abspath = getcwd()
   dname = os.path.dirname(abspath)
   os.chdir(dname)

set_path()
 

In [3]:
df_train = pd.read_pickle('data/processed/train_reduced_df.pkl')
df_val = pd.read_pickle('data/processed/val_reduced_df.pkl')

### Encoding with TFIDF

In [4]:
df_train.head(3)

,content,year,venues_le,authId_enc,new_content
0,detect linguist idiosyncrat interest autism di...,2014,58,1571,detect linguist idiosyncrat interest autism di...
1,bigram bilstm neural network sequenti metaphor...,2018,122,1250,bigram bilstm neural network sequenti metaphor...
2,factual effici integr relev fact visual questi...,2021,5,4134,factual effici integr relev fact visual questi...


In [5]:
xtrain, ytrain = df_train['new_content'].values, df_train['authId_enc'].values
xval, yval = df_val['new_content'].values, df_val['authId_enc'].values

In [6]:
tfvectorizer = TfidfVectorizer()
xtrain_tf = tfvectorizer.fit_transform(xtrain)
xval_tf = tfvectorizer.transform(xval)


In [7]:
max(df_train['authId_enc'])

5624

In [8]:
xval_tf.shape,xtrain_tf.shape

((100, 7982), (12029, 7982))

In [9]:
yval.shape, ytrain.shape

((100,), (12029,))

### Experiment with randomForest

In [14]:
classifier = RandomForestClassifier()

classifier.fit(xtrain_tf,ytrain)

KeyboardInterrupt: 

In [ ]:
y_pred = classifier.predict(xval_tf)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(yval, y_pred)

### MLP

In [10]:
from tensorflow.keras import models
from tensorflow.keras import layers
nb_words = 8000
def build_model():
    ### PUT YOUR CODE HERE ###
    model = models.Sequential()
    model.add(layers.Dense(256, activation='relu', input_shape=(xtrain_tf.shape[1],)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(5624, activation='sigmoid'))
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model
    ### END ###

In [11]:
mlp = build_model()
original_hist = mlp.fit(xtrain_tf,
                          ytrain,
                          epochs=20,
                          batch_size=128,
                          validation_data=(xval_tf, yval),
                          verbose=0)

InvalidArgumentError: {{function_node __wrapped__SerializeManySparse_device_/job:localhost/replica:0/task:0/device:CPU:0}} indices[1] = [0,1776] is out of order. Many sparse ops require sorted indices.
    Use `tf.sparse.reorder` to create a correctly ordered copy.

 [Op:SerializeManySparse]